In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
DATA_LOC = "/content/drive/MyDrive/Rhexis/datasets/test_pulls"
REPO_LOC = "/content/drive/MyDrive/Stanford/rhexis-trajectory"

In [ ]:
# Import libraries
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import sys
sys.path.insert(0,f"{REPO_LOC}/Trajectory_Classification")
from utils import *
%load_ext autoreload
%autoreload 2

# Load data

In [ ]:
names, path_dfs, labels, sizes = load_all_pulls(DATA_LOC)

In [ ]:
print(names)
#print(path_dfs)